# Primeros pasos para realizar la clasificación supervisada obtenida de un Raster Landcover de la Región de Aysen (2018)


Script trabajado por Francisco Albornoz (2021-04-12) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

Para crear un nuevo ambiente de trabajo realizamos este comando desde nuestro terminal:
    
- `conda create -n gee python`
- `conda activate gee`
- `conda install mamba -c conda-forge`
- `mamba install geemap xarray_leaflet -c conda-forge`

Para correr aquellas librerías no encontradas instalamos con el siguiente comando: (correr borrando # o digitalizando ALT+C)

In [ ]:
# !pip install xarray_leaflet
# !pip install geemap

Importamos las librerías a utilizar dentro del siguiente script de Jupyter Notebooks:

  -  os
  - geemap
  - xarray_leaflet
  - ee
  - rasterio
  - numpy
  - matplotlib
  

In [ ]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Importar un mapa interactivo

In [ ]:
Map = geemap.Map()
Map

Insertamos las coordenadas de trabajo, y agregamos una imagen Landsat

In [ ]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")

Verificación de datos de la imagen landsat

In [ ]:
ee.Date(landsat.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [ ]:
landsat.get('CLOUD_COVER').getInfo()

## Importar rasters locales

Debemos localizar nuestro raster a subir, verificamos nuestra carpeta de trabajo:

In [ ]:
%pwd

In [ ]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 

In [ ]:
# verificar en el sistema del pc cual es la ruta
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [ ]:
out_dir

In [ ]:
raster_aysen

In [ ]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

In [ ]:
# os.listdir()

In [ ]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [ ]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [ ]:
raster.bounds

In [ ]:
raster.crs

In [ ]:
raster

## Obteniendo los valores para el Training Datasets

In [ ]:
points = landcover.sample(**{
    'region': landsat.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map


In [ ]:
print(points.size().getInfo())

In [ ]:
print(points.first().getInfo())

## Entrenando el modelo 

In [ ]:
# Usar las bandas para la predicción.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #agregar NDVI


# Esta propiedad guarda los valores del landcover 2018
label = 'b1'

# Overlay the points on the imagery to get training.
training = landsat.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

## Clasificando la Imagen 

In [ ]:
# Clasificar las imagenes con las mismas bandas usadas para el entrenamiento
result = landsat.select(bands).classify(trained)

# # Viendo los cluster con colores 
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

In [ ]:
result.getInfo()

## Categorizando según las clases 

In [ ]:
landcover.getInfo()

In [ ]:
class_values = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16]

In [ ]:
class_palette = ['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94']


In [ ]:
landcoverf = result.set('classification_class_values', class_values)
landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcoverf, {}, 'Land cover Final' )
Map